In [1]:
# Setup

import os
import sys
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import Quorum
from keras.datasets import cifar100
from keras.utils import np_utils
from keras import backend as K
from keras.activations import relu
from keras.callbacks import ModelCheckpoint

def pushln(line):
    sys.stdout.write(line)
    sys.stdout.flush()
    
def step(X):
    return K.switch(X <= 0, 0, 1)

def prepare_input_data(X_train, X_test):
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    return X_train, X_test

def prepare_output_data(y_train, y_test):
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return y_train, y_test

def get_af(name):
    af = name.lower()
    
    if af == "relu":
        return Activation('relu')
    elif af == "pap":
        return Quorum([relu, step])
    else:
        raise RuntimeError("Unrecognized activation function: {}".format(name))

def add_convolutional_layers(model, activation_name, n, filter_size, window, dropout=0.25, stack_finished=True):
    for i in range(n):
        model.add(Convolution2D(filter_size, window, window, border_mode="same"))
        model.add(get_af(activation_name))
    if stack_finished:
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))

Using Theano backend.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [ ]:
# Script

af = "pap" #pap, relu

(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train, X_test = prepare_input_data(X_train, X_test)
y_train, y_test = prepare_output_data(y_train, y_test)

model = Sequential()

# 1 x 384 x 3
model.add(Convolution2D(384, 3, 3, border_mode='same',
                        input_shape=(3, 32, 32)))
model.add(get_af(af))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 1 x 384 x 1
add_convolutional_layers(model, af, 1, 384, 1, stack_finished=False)
# 2 x 640 x 2
add_convolutional_layers(model, af, 2, 640, 2)

# 1 x 640 x 3
add_convolutional_layers(model, af, 1, 640, 3, stack_finished=False)
# 3 x 768 x 3
add_convolutional_layers(model, af, 3, 768, 3)

# 1 x 768 x 3
add_convolutional_layers(model, af, 1, 768, 3, stack_finished=False)
# 2 x 896 x 3
add_convolutional_layers(model, af, 2, 896, 3)

# 1 x 896 x 3
add_convolutional_layers(model, af, 1, 896, 3, stack_finished=False)
# 2 x 1024 x 3
add_convolutional_layers(model, af, 2, 1024, 3)

# 1 x 1024 x 3
add_convolutional_layers(model, af, 1, 1024, 3, stack_finished=False)
# 2 x 1152 x 3
add_convolutional_layers(model, af, 2, 1152, 3, stack_finished=False)

# 1 x 100 x FC
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(4096))
model.add(get_af(af))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(get_af(af))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('softmax'))

pushln("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='sgd')
pushln("finished.\n")

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

pushln("Fitting ImageDataGenerator...")
datagen.fit(X_train)
pushln("finished...\n")
mc = ModelCheckpoint("weights_"+af, save_best_only=True, verbose=1)
pushln("Beginning training...\n")
hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), verbose=1,
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=200, show_accuracy=True,
                        validation_data=(X_test, y_test),
                        nb_worker=1, callbacks=[mc])

Compiling model...finished.
Fitting ImageDataGenerator...finished...Beginning training...
